<a href="https://colab.research.google.com/github/3irsari/Rasa_Projects/blob/main/Conversation_AI_with_the_Rasa_Stack.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Conversation AI with the Rasa Stack

In this notebook you'll build a complete chatbot with Rasa Stack, without having to install anything on your computer. This is a relatively simple bot, that just asks you about your mood and tries to cheer you up if you're feeling a bit down.



*   Part 1: We'll start with a basic bot that can handle multi-turn dialogues but not understand natural language like "I'm doing well"
*   Part 2: We'll add the abilitiy to understand natural language so that the bot can generalise to similar inputs that it hasn't seen before
*   Part 3: We'll give you further resources so you can build your own.


## Part 1: Having a basic conversation

### Get Rasa Core
First we'll have to install Rasa Core on the server running this notebook.

In [ ]:
!pip install rasa_core;

import logging, io, json, warnings
logging.basicConfig(level="INFO")
warnings.filterwarnings('ignore')

    100% |████████████████████████████████| 204kB 23.4MB/s 
    100% |████████████████████████████████| 665kB 18.3MB/s 
    100% |████████████████████████████████| 71kB 25.9MB/s 
    100% |████████████████████████████████| 153kB 28.7MB/s 
    100% |████████████████████████████████| 307kB 27.7MB/s 
    100% |████████████████████████████████| 4.9MB 5.3MB/s 
    100% |████████████████████████████████| 58.4MB 576kB/s 
    100% |████████████████████████████████| 235kB 21.8MB/s 
    100% |████████████████████████████████| 71kB 25.0MB/s 
    100% |████████████████████████████████| 61kB 22.4MB/s 
    100% |████████████████████████████████| 51kB 21.4MB/s 
    100% |████████████████████████████████| 983kB 14.8MB/s 
    100% |████████████████████████████████| 61kB 22.8MB/s 
    100% |████████████████████████████████| 112kB 31.3MB/s 
    100% |████████████████████████████████| 256kB 21.6MB/s 
    100% |████████████████████████████████| 51kB 21.4MB/s 
    100% |████████████████████████████████| 81k

### Understanding Conversations through Stories

Lets us get started by writing a few stories, which are example conversations that Rasa Core will learn from.

The format works like this:

A story starts with `##` and you can give it a name.
- **Intents:** Lines that start with `*` are messages sent by the user. Intent (and the entities) that represent what the user *means*.

- **Actions:** Lines that start with `-` are *actions* taken by your bot. In this case all of our actions are just messages sent back to the user, like `utter_greet`, but in general an action can do anything, including calling an API and interacting with the outside world.

In [ ]:
stories_md = """
## happy path               <!-- name of the story - just for debugging -->
* greet
  - utter_greet
* mood_great               <!-- user utterance, in format intent[entities] -->
  - utter_happy
* mood_affirm
  - utter_happy
* mood_affirm
  - utter_goodbye


## sad path 1               <!-- this is already the start of the next story -->
* greet
  - utter_greet             <!-- action the bot should execute -->
* mood_unhappy
  - utter_cheer_up
  - utter_did_that_help
* mood_affirm
  - utter_happy



## sad path 2
* greet
  - utter_greet
* mood_unhappy
  - utter_cheer_up
  - utter_did_that_help
* mood_deny
  - utter_goodbye

## strange user
* mood_affirm
  - utter_happy
* mood_affirm
  - utter_unclear

## say goodbye
* goodbye
  - utter_goodbye

"""

%store stories_md > stories.md

Writing 'stories_md' (str) to file 'stories.md'.


### Defining a Domain for your agent

The domain specifies the universe that your bot lives in. All of the intents and actions that show up in your stories should be listed along with templates, which contain the messages your bot can send back.

In [ ]:
domain_yml = """
intents:
  - greet
  - goodbye
  - mood_affirm
  - mood_deny
  - mood_great
  - mood_unhappy

actions:
  - utter_greet
  - utter_cheer_up
  - utter_did_that_help
  - utter_happy
  - utter_goodbye
  - utter_unclear

templates:
  utter_greet:
  - text: "Hey! How are you?"

  utter_cheer_up:
  - text: "Here is something to cheer you up:"
    text: "https://i.imgur.com/nGF1K8f.jpg"

  utter_did_that_help:
  - text: "Did that help you?"

  utter_unclear:
  - text: "I am not sure what you are aiming for."

  utter_happy:
  - text: "Great carry on!"

  utter_goodbye:
  - text: "Bye"
"""

%store domain_yml > domain.yml

Writing 'domain_yml' (str) to file 'domain.yml'.


### Training your Dialogue Model

Now comes the fun part! We're going to show Rasa Core the stories we wrote above, and train a model on these examples.

In this case, the model is a neural network implemented in Keras which learns to predict which action to take next. The default architecture is based on an LSTM.

In [ ]:
from rasa_core.policies.keras_policy import KerasPolicy
from rasa_core.agent import Agent

agent = Agent('domain.yml', policies=[KerasPolicy()])
training_data = agent.load_data('stories.md')
agent.train(
        training_data,
        validation_split=0.0
)

agent.persist('models/dialogue')

### Starting up the bot
Since we persisted our bot in `models/dialogue`, we can now load it into our agent and start our bot.

In [ ]:
from rasa_core.agent import Agent
agent = Agent.load('models/dialogue')

### Talking to the bot (no NLU)

Now that we trained the dialogue model, basically the conversational flow, the bot will only understand **structured input**. We still have to implement a natural language understanding.

In [ ]:
print("Your bot is ready to talk! Type your messages here or send 'stop'")
while True:
    a = input()
    if a == 'stop':
        break
    responses = agent.handle_text(a)
    for response in responses:
        print(response["text"])

## Part 2: Adding Natural Language Understanding
So far, our bot doesn't understand regular sentences with natural language yet.
To do that, we're going to build a language understanding model with Rasa NLU.

### Installing NLU
Let's get Rasa NLU on board.


In [ ]:
!apt-get -qq install -y graphviz libgraphviz-dev pkg-config && pip install pygraphviz;
!pip install rasa_nlu[spacy];
!python -m spacy download en;

### Language Understanding


Lets create some training data here, grouping user messages by their `intent`. The intent basically describes what the messages **mean**.

In [ ]:
nlu_md = """
## intent:greet
- hey
- hello
- hi
- hello there
- good morning
- good evening
- moin
- hey there
- let's go
- hey dude
- goodmorning
- goodevening
- good afternoon

## intent:goodbye
- cu
- good by
- cee you later
- good night
- good afternoon
- bye
- goodbye
- have a nice day
- see you around
- bye bye
- see you later

## intent:mood_affirm
- yes
- indeed
- of course
- that sounds good
- correct

## intent:mood_deny
- no
- never
- I don't think so
- don't like that
- no way
- not really

## intent:mood_great
- perfect
- very good
- great
- amazing
- feeling like a king
- wonderful
- I am feeling very good
- I am great
- I am amazing
- I am going to save the world
- super
- extremely good
- so so perfect
- so good
- so perfect

## intent:mood_unhappy
- my day was horrible
- I am sad
- I don't feel very well
- I am disappointed
- super sad
- I'm so sad
- sad
- very sad
- unhappy
- bad
- very bad
- awful
- terrible
- not so good
- not very good
- extremly sad
- so saad
- so sad
"""

%store nlu_md > nlu.md

### Train the Rasa NLU Model

We're going to train a model to recognise these intents, so that when we send a message like "hello" to our bot, it will recognise this as a "greet" intent.

In [ ]:
from rasa_nlu.training_data import load_data
from rasa_nlu.config import RasaNLUModelConfig
from rasa_nlu.model import Trainer

training_data = load_data('nlu.md')
pipeline = [{"name": "nlp_spacy"},
            {"name": "tokenizer_spacy"},
            {"name": "intent_featurizer_spacy"},
            {"name": "intent_classifier_sklearn"}]
trainer = Trainer(RasaNLUModelConfig({"pipeline": pipeline}))
interpreter = trainer.train(training_data)
model_directory = trainer.persist('./projects/default/')

### Starting up the bot (with NLU)

Now that we've trained the dialogue **and** language understanding models and saved them, we can start up an `Agent` which will handle conversations for us.

In [ ]:
from rasa_core.agent import Agent
agent = Agent.load('models/dialogue', interpreter=model_directory)

### Talking to the Bot (with NLU)

We can start talking to the bot in natural language.

In [ ]:
print("Your bot is ready to talk! Type your messages here or send 'stop'")
while True:
    a = input()
    if a == 'stop':
        break
    responses = agent.handle_text(a)
    for response in responses:
        print(response["text"])


### Pro Tip: Visualising the Training Data

You can visualise the stories to get a sense of how the conversations go. This is usually a good way to see if there are any stories which don't make sense


In [ ]:
from rasa_core.agent import Agent
from IPython.display import IFrame

agent = Agent.load('models/dialogue')
agent.visualize("stories.md", "story_graph.html", max_history=2)

IFrame(src="./story_graph.html", width=1000, height=800)

## Part 3: Next steps

Great job! You've built your first bot that uses Machine Learning to manage dialogues. As a next step, we'd suggest you start building your own bot. Here are a few useful links:



*   Install the Rasa Stack on your machine [here](https://core.rasa.ai/installation.html)
*   Learn how to bootstrap training data with the [interactive learning tutorial](https://core.rasa.ai/tutorial_interactive_learning.html)
*  Breaking down the Rasa NLU training model - [here](https://medium.com/@gauravgpunjabi/getting-started-with-machine-learning-ea0dc29e3ff6)
* Understanding Natural Language Processing - [here](https://medium.com/@ageitgey/natural-language-processing-is-fun-9a0bff37854e)